In [1]:
# Import packages
from __future__ import print_function
import keras
from keras.datasets import cifar100
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.applications import ResNet50, MobileNetV2, EfficientNetB0

import os
import matplotlib.pyplot as plt
%matplotlib inline

import keras.backend as K
import tensorflow as tf

import json
import time

In [2]:
import cv2
import numpy as np

from keras.datasets import cifar100
from keras import backend as KK
from keras.utils import np_utils

#nb_train_samples = 3000 # 3000 training samples
#nb_valid_samples = 100 # 100 validation samples
num_classes = 100

def load_cifar100_data(img_rows, img_cols):

    # Load cifar10 training and validation sets
    (X_train, Y_train), (X_valid, Y_valid) = cifar100.load_data()
    nb_train_samples = len(X_train)
    nb_valid_samples = len(X_valid)

    # Resize trainging images
#     if KK.image_dim_ordering() == 'th':
#         X_train = np.array([cv2.resize(img.transpose(1,2,0), (img_rows,img_cols)).transpose(2,0,1) for img in X_train[:nb_train_samples,:,:,:]])
#         X_valid = np.array([cv2.resize(img.transpose(1,2,0), (img_rows,img_cols)).transpose(2,0,1) for img in X_valid[:nb_valid_samples,:,:,:]])
#     else:
    X_train = np.array([cv2.resize(img, (img_rows,img_cols)) for img in X_train[:nb_train_samples,:,:,:]])
    X_valid = np.array([cv2.resize(img, (img_rows,img_cols)) for img in X_valid[:nb_valid_samples,:,:,:]])

    # Transform targets to keras compatible format
#     Y_train = np_utils.to_categorical(Y_train[:nb_train_samples], num_classes)
#     Y_valid = np_utils.to_categorical(Y_valid[:nb_valid_samples], num_classes)

    return X_train, Y_train, X_valid, Y_valid


## Define custom loss functions and metrics

In [3]:
def superclass1_loss(y_true,y_pred):
    # This is the cross entropy loss at the superclass level (20 superclasses)
    y_true = tf.clip_by_value(y_true, 1e-16, 1-(1e-16))
    y_pred = tf.clip_by_value(y_pred, 1e-16, 1-(1e-16))
    part1 = [[4, 30, 55, 72, 95],[1, 32, 67, 73, 91],[54, 62, 70, 82, 92],
             [9, 10, 16, 28, 61],[0, 51, 53, 57, 83],[22, 39, 40, 86, 87],
             [5, 20, 25, 84, 94],[6, 7, 14, 18, 24], [3, 42, 43, 88, 97],
             [12, 17, 37, 68, 76], [23, 33, 49, 60, 71], [15, 19, 21, 31, 38],
             [34, 63, 64, 66, 75], [26, 45, 77, 79, 99], [2, 11, 35, 46, 98],
             [27, 29, 44, 78, 93], [36, 50, 65, 74, 80], [47, 52, 56, 59, 96],
             [8, 13, 48, 58, 90], [41, 69, 81, 85, 89]]
    loss1=0
    for sublist in part1:
        ytt_temp = K.sum(tf.gather(y_true,sublist,axis=1),axis=-1)
        ypt_temp = K.sum(tf.gather(y_pred,sublist,axis=1),axis=-1)
        loss1+=ytt_temp*(-1)*K.log(ypt_temp)
    return loss1

In [4]:
def superclass2_loss(y_true,y_pred):
    # This is the cross entropy loss at the level 1 up from superclass (8 choices)
    # 4 animals (human, mammal, water, other)
    # plant, landscape, and 2 objects (vehicle, other)
    y_true = tf.clip_by_value(y_true, 1e-16, 1-(1e-16))
    y_pred = tf.clip_by_value(y_pred, 1e-16, 1-(1e-16))
    part2 = [[4, 30, 55, 72, 95, 1, 32, 67, 73, 91],
             [54, 62, 70, 82, 92, 0, 51, 53, 57, 83, 47, 52, 56, 59, 96],
             [6, 7, 14, 18, 24, 26, 45, 77, 79, 99, 27, 29, 44, 78, 93],
             [3, 42, 43, 88, 97, 15, 19, 21, 31, 38, 34, 63, 64, 66, 75, 36, 50, 65, 74, 80],
             [2, 11, 35, 46, 98],
             [23, 33, 49, 60, 71, 12, 17, 37, 68, 76],
             [9, 10, 16, 28, 61, 22, 39, 40, 86, 87, 5, 20, 25, 84, 94],
             [8, 13, 48, 58, 90, 41, 69, 81, 85, 89]]

    loss2=0
    for sublist in part2:
        ytt_temp = K.sum(tf.gather(y_true,sublist,axis=1),axis=-1)
        ypt_temp = K.sum(tf.gather(y_pred,sublist,axis=1),axis=-1)
        loss2+=ytt_temp*(-1)*K.log(ypt_temp)
    return loss2

In [5]:
def superclass3_loss(y_true,y_pred):
    # This is the cross entropy loss at the level 2 up from superclass (4 choices)
    # animal, plant, landscape, object (manmade)
    y_true = tf.clip_by_value(y_true, 1e-16, 1-(1e-16))
    y_pred = tf.clip_by_value(y_pred, 1e-16, 1-(1e-16))
    part3 = [[4, 30, 55, 72, 95, 1, 32, 67, 73, 91, 6, 7, 14, 18, 24, 26, 45, 77, 79,
              99, 27, 29, 44, 78, 93, 3, 42, 43, 88, 97, 15, 19, 21, 31, 38, 34, 63, 64,
              66, 75, 36, 50, 65, 74, 80, 2, 11, 35, 46, 98],
             [54, 62, 70, 82, 92, 0, 51, 53, 57, 83, 47, 52, 56, 59, 96],
             [23, 33, 49, 60, 71, 12, 17, 37, 68, 76],
             [9, 10, 16, 28, 61, 22, 39, 40, 86, 87, 5, 20, 25, 84, 94, 8, 13, 48, 58,
              90, 41, 69, 81, 85, 89]]
    loss3=0
    for sublist in part3:
        ytt_temp = K.sum(tf.gather(y_true,sublist,axis=1),axis=-1)
        ypt_temp = K.sum(tf.gather(y_pred,sublist,axis=1),axis=-1)
        loss3+=ytt_temp*(-1)*K.log(ypt_temp)
    return loss3

In [6]:
def struct1_loss(y_true,y_pred):
    # This is an equal blend of superclass_loss and "normal" (class-level) loss
    y_true = tf.clip_by_value(y_true, 1e-16, 1-(1e-16))
    y_pred = tf.clip_by_value(y_pred, 1e-16, 1-(1e-16))
    loss0 = K.sum(y_true*(-1)*K.log(y_pred), axis=-1)
    loss1 = superclass1_loss(y_true, y_pred)
    return .5*loss0+.5*loss1

In [7]:
def struct2_loss(y_true,y_pred):
    # This is an equal blend of superclass2_loss, superclass1_loss 
    # and "normal" (class-level) loss
    y_true = tf.clip_by_value(y_true, 1e-16, 1-(1e-16))
    y_pred = tf.clip_by_value(y_pred, 1e-16, 1-(1e-16))
    loss0 = K.sum(y_true*(-1)*K.log(y_pred), axis=-1)
    loss1 = superclass1_loss(y_true, y_pred)
    loss2 = superclass2_loss(y_true, y_pred)
    return (1/3)*loss0+(1/3)*loss1+(1/3)*loss2

In [8]:
def struct3_loss(y_true,y_pred):
    # This is an equal blend of superclass3_loss,superclass2_loss, 
    # superclass1_loss and "normal" (class-level) loss
    y_true = tf.clip_by_value(y_true, 1e-16, 1-(1e-16))
    y_pred = tf.clip_by_value(y_pred, 1e-16, 1-(1e-16))
    loss0 = K.sum(y_true*(-1)*K.log(y_pred), axis=-1)
    loss1 = superclass1_loss(y_true, y_pred)
    loss2 = superclass2_loss(y_true, y_pred)
    loss3 = superclass3_loss(y_true, y_pred)
    return .25*loss0+.25*loss1+.25*loss2+.25*loss3

In [9]:
def super1_acc(y_true, y_pred):
    table = tf.lookup.StaticHashTable(
    initializer=tf.lookup.KeyValueTensorInitializer(
        keys=tf.constant([4, 30, 55, 72, 95, 1, 32, 67, 73, 91, 54, 62, 70, 82, 92, 9, 10, 16, 28, 61, 0, 51, 53, 57, 83, 
                          22, 39, 40, 86, 87, 5, 20, 25, 84, 94, 6, 7, 14, 18, 24, 3, 42, 43, 88, 97, 12, 17, 37, 68, 76, 
                          23, 33, 49, 60, 71, 15, 19, 21, 31, 38, 34, 63, 64, 66, 75, 26, 45, 77, 79, 99, 2, 11, 35, 46, 98,
                          27, 29, 44, 78, 93, 36, 50, 65, 74, 80, 47, 52, 56, 59, 96, 8, 13, 48, 58, 90, 41, 69, 81, 85, 89]),
        values=tf.constant([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 
                            7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 
                            13, 13, 13, 13, 13, 14, 14, 14, 14, 14,
                            15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 17, 17, 17, 17, 17, 18, 18, 18, 18, 18, 19, 19, 19, 19, 19]),
    ),
    default_value=tf.constant(-1),
    name="superclass_index"
    )
    yt_hard = tf.argmax(y_true, axis=1)
    yp_hard = tf.argmax(y_pred, axis=1)
    
    ytc = table.lookup(tf.cast(yt_hard, tf.int32))
    ypc = table.lookup(tf.cast(yp_hard, tf.int32))
    return K.sum(tf.cast(tf.equal(ytc,ypc),tf.int32))/tf.shape(ytc)

In [10]:
def super2_acc(y_true, y_pred):
    table = tf.lookup.StaticHashTable(
    initializer=tf.lookup.KeyValueTensorInitializer(
        keys=tf.constant([4, 30, 55, 72, 95, 1, 32, 67, 73, 91, 54, 62, 70, 82, 92, 9, 10, 16, 28, 61, 0, 51, 53, 57, 83, 
                          22, 39, 40, 86, 87, 5, 20, 25, 84, 94, 6, 7, 14, 18, 24, 3, 42, 43, 88, 97, 12, 17, 37, 68, 76, 
                          23, 33, 49, 60, 71, 15, 19, 21, 31, 38, 34, 63, 64, 66, 75, 26, 45, 77, 79, 99, 2, 11, 35, 46, 98,
                          27, 29, 44, 78, 93, 36, 50, 65, 74, 80, 47, 52, 56, 59, 96, 8, 13, 48, 58, 90, 41, 69, 81, 85, 89]),
        values=tf.constant([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 
                            2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 
                            3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 
                            6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]),
    ),
    default_value=tf.constant(-1),
    name="superclass2_index"
    )
    yt_hard = tf.argmax(y_true, axis=1)
    yp_hard = tf.argmax(y_pred, axis=1)
    
    ytc = table.lookup(tf.cast(yt_hard, tf.int32))
    ypc = table.lookup(tf.cast(yp_hard, tf.int32))
    return K.sum(tf.cast(tf.equal(ytc,ypc),tf.int32))/tf.shape(ytc)

In [11]:
def super3_acc(y_true, y_pred):
    table = tf.lookup.StaticHashTable(
    initializer=tf.lookup.KeyValueTensorInitializer(
        keys=tf.constant([4, 30, 55, 72, 95, 1, 32, 67, 73, 91, 54, 62, 70, 82, 92, 9, 10, 16, 28, 61, 0, 51, 53, 57, 83, 
                          22, 39, 40, 86, 87, 5, 20, 25, 84, 94, 6, 7, 14, 18, 24, 3, 42, 43, 88, 97, 12, 17, 37, 68, 76, 
                          23, 33, 49, 60, 71, 15, 19, 21, 31, 38, 34, 63, 64, 66, 75, 26, 45, 77, 79, 99, 2, 11, 35, 46, 98,
                          27, 29, 44, 78, 93, 36, 50, 65, 74, 80, 47, 52, 56, 59, 96, 8, 13, 48, 58, 90, 41, 69, 81, 85, 89]),
        values=tf.constant([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                            0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                            1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 
                            3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]),
    ),
    default_value=tf.constant(-1),
    name="superclass2_index"
    )
    yt_hard = tf.argmax(y_true, axis=1)
    yp_hard = tf.argmax(y_pred, axis=1)
    
    ytc = table.lookup(tf.cast(yt_hard, tf.int32))
    ypc = table.lookup(tf.cast(yp_hard, tf.int32))
    return K.sum(tf.cast(tf.equal(ytc,ypc),tf.int32))/tf.shape(ytc)

## Begin Training loop

In [12]:
# Set some parameters
batch_size = 40
num_classes = 100
epochs = 300
data_augmentation = True
train_set_size = 1000
num_trials = 3
loss_fn_to_use = struct3_loss
lr = 0.002
timestamp = int(time.time())

In [13]:
def new_model(num_classes):


    base_model = 0
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))

    x = base_model.output
    x_newfc = x = GlobalAveragePooling2D()(x)
    x_newfc = Dense(num_classes, activation='softmax', name='fc10')(x_newfc)
    model = Model(inputs=base_model.input, outputs=x_newfc)

    for layer in base_model.layers:
        layer.trainable = False

    return model

In [14]:
print('loading data')
(x_train_full, y_train_full, x_test, y_test_simple) = load_cifar100_data(224, 224)
print('x_train shape:', x_train_full.shape)

for i in range(num_trials):
    # The data, split between train and test sets:
    print('trial {}'.format(i))
    x_train = x_train_full[i*train_set_size:(i+1)*train_set_size,:,:,:]
    y_train_simple = y_train_full[i*train_set_size:(i+1)*train_set_size,:]
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # Convert class vectors to binary class matrices.
    y_train = keras.utils.to_categorical(y_train_simple, num_classes)
    y_test = keras.utils.to_categorical(y_test_simple, num_classes)

    model = 0
    model = new_model(num_classes)
    
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255

    sgd = keras.optimizers.SGD(lr=lr, decay=1e-6, momentum=0.9, nesterov=True)

    model.compile(loss=loss_fn_to_use,
                  optimizer=sgd,
                  metrics=['accuracy',super1_acc,super2_acc, super3_acc,
                           'categorical_crossentropy',superclass1_loss, superclass2_loss,superclass3_loss,
                           struct1_loss, struct2_loss, struct3_loss])

    
    if not data_augmentation:
        print('Not using data augmentation.')
        history=model.fit(x_train, y_train,
                  batch_size=batch_size,
                  epochs=epochs,
                  validation_data=(x_test, y_test),
                  shuffle=True)

    else:
        print('Using real-time data augmentation.')
        # This will do preprocessing and realtime data augmentation:
        datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            zca_epsilon=1e-06,  # epsilon for ZCA whitening
            rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
            # randomly shift images horizontally (fraction of total width)
            width_shift_range=0.1,
            # randomly shift images vertically (fraction of total height)
            height_shift_range=0.1,
            shear_range=0.,  # set range for random shear
            zoom_range=0.,  # set range for random zoom
            channel_shift_range=0.,  # set range for random channel shifts
            # set mode for filling points outside the input boundaries
            fill_mode='nearest',
            cval=0.,  # value used for fill_mode = "constant"
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False,  # randomly flip images
            # set rescaling factor (applied before any other transformation)
            rescale=None,
            # set function that will be applied on each input
            preprocessing_function=None,
            # image data format, either "channels_first" or "channels_last"
            data_format=None,
            # fraction of images reserved for validation (strictly between 0 and 1)
            validation_split=0.0)

        # Compute quantities required for feature-wise normalization
        # (std, mean, and principal components if ZCA whitening is applied).
        datagen.fit(x_train)

        # Fit the model on the batches generated by datagen.flow().
        history=model.fit(datagen.flow(x_train, y_train,
                                         batch_size=batch_size),
                            epochs=epochs,
                            validation_data=(x_test, y_test),
                            workers=4, shuffle=True, verbose=2)



    str_name = 'CIFAR100_results'+'_'+str(timestamp)+'_mnet_s3_loss_ts_'+str(train_set_size)+'_trial_'+str(i)
    str_name

    with open(str_name, 'w') as outfile:
        json.dump(history.history, outfile)


loading data
x_train shape: (50000, 224, 224, 3)
trial 2
1000 train samples
10000 test samples
Shape: (None, 7, 7, 1280)
Using real-time data augmentation.
Epoch 1/300
25/25 - 196s - loss: 2.8194 - accuracy: 0.0120 - super1_acc: 0.0590 - super2_acc: 0.1350 - super3_acc: 0.3130 - categorical_crossentropy: 4.9850 - superclass1_loss: 3.0476 - superclass2_loss: 2.0340 - superclass3_loss: 1.2109 - struct1_loss: 4.0163 - struct2_loss: 3.3555 - struct3_loss: 2.8194 - val_loss: 2.7156 - val_accuracy: 0.0213 - val_super1_acc: 0.1003 - val_super2_acc: 0.1785 - val_super3_acc: 0.3443 - val_categorical_crossentropy: 4.8315 - val_superclass1_loss: 2.9389 - val_superclass2_loss: 1.9484 - val_superclass3_loss: 1.1438 - val_struct1_loss: 3.8852 - val_struct2_loss: 3.2396 - val_struct3_loss: 2.7156
Epoch 2/300
25/25 - 184s - loss: 2.6403 - accuracy: 0.0400 - super1_acc: 0.1330 - super2_acc: 0.2260 - super3_acc: 0.3590 - categorical_crossentropy: 4.6834 - superclass1_loss: 2.8641 - superclass2_loss: 1.8

KeyboardInterrupt: 